In [1]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import pandas as pd
import random

import torch
import torch.nn as nn
from statsmodels.tsa.seasonal import seasonal_decompose #이동평균을 이용해서 na가 나옴

import gc
import math


import gc
import math
import os.path
import time

import matplotlib.pyplot as plt
from datetime import timedelta, datetime
from dateutil import parser
from tqdm import tqdm
import copy
import warnings
warnings.filterwarnings("ignore")


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(device)

cpu


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def df2d_to_array3d(df_2d):
    # 입력 받은 2차원 데이터 프레임을 3차원 numpy array로 변경하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    return array_3d

def split_data(stock, lookback):
    data_raw = stock.to_numpy() # df convert to numpy array
    data = []
    
    # create all possible sequences of length seq_len
    for index in range(len(data_raw) - lookback): 
        data.append(data_raw[index: index + lookback])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:,:-1,:]
    y_train = data[:,-1,:]
    
    return [x_train, y_train]

def array_to_submission(pred_array):
    # 입력 x_arrry와 출력 pred_arry를 통해서 
    # buy_quantitiy와 sell_time을 결정
    submission = pd.DataFrame(np.zeros([pred_array.shape[0],2], np.int64),
                columns = ['buy_quantity', 'sell_time'])
    submission = submission.reset_index()
    submission.loc[:, 'buy_quantity'] = 0.1
    
    buy_price = []
    for idx, sell_time in enumerate(np.argmax(pred_array, axis = 1)):
        buy_price.append(pred_array[idx, sell_time])
    buy_price = np.array(buy_price)
    # 115% 이상 상승한하고 예측한 sample에 대해서만 100% 매수
    submission.loc[:, 'buy_quantity'] = (buy_price > 0.95) * 0.1
    # 모델이 예측값 중 최대 값에 해당하는 시간에 매도
    submission['sell_time'] = np.argmax(pred_array, axis = 1)
    submission.columns = ['sample_id','buy_quantity', 'sell_time']
    return submission

def df2d_to_answer(df_2d):
    # valid_y_df로부터
    # open 가격 정보가 포함된
    # [샘플 수, 120분] 크기의 
    # 2차원 array를 반환하는 함수
    feature_size = df_2d.iloc[:,2:].shape[1]
    time_size = len(df_2d.time.value_counts())
    sample_size = len(df_2d.sample_id.value_counts())
    sample_index = df_2d.sample_id.value_counts().index
    array_2d = df_2d.open.values.reshape([sample_size, time_size])
    sample_index = list(sample_index)
    return array_2d, sample_index


def COIN(y_df, submission, df2d_to_answer = df2d_to_answer):
    # 2차원 데이터프레임에서 open 시점 데이터만 추출하여 array로 복원
    # sample_id정보를 index에 저장
    y_array, index = df2d_to_answer(y_df)
    
    # index 기준으로 submission을 다시 선택
    submission = submission.set_index(submission.columns[0])
    submission = submission.iloc[index, :]    
    
    # 초기 투자 비용은 10000 달러
    total_momey      = 10000 # dolors
    total_momey_list = []
    
    # 가장 처음 sample_id값
    start_index = submission.index[0]
    for row_idx in submission.index:
        sell_time  = submission.loc[row_idx, 'sell_time']
        buy_price  = y_array[row_idx - start_index, 0]
        sell_price = y_array[row_idx - start_index, sell_time]
        buy_quantity = submission.loc[row_idx, 'buy_quantity'] * total_momey
        residual = total_momey - buy_quantity
        ratio = sell_price / buy_price
        total_momey = buy_quantity * ratio * 0.9995 * 0.9995 + residual        
        total_momey_list.append(total_momey)
        
    return total_momey, total_momey_list

In [5]:
data_path='/content/drive/MyDrive/Colab Notebooks/Bitcoin_Trading/data'
test_x_df = pd.read_csv(data_path  + "/test_x_df.csv")
test_x_df=test_x_df.drop(['volume', 'quote_av','trades','tb_base_av','tb_quote_av'], axis=1)

In [6]:
test_x_array = df2d_to_array3d(test_x_df)

print(f'''
test_x_array {test_x_array.shape}
''')


test_x_array (535, 1380, 5)



## Model 설정

In [7]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

lookback=40 #나중에 model 학습시켰을 때 뭐 했는지 torch.save로 함꼐 저장하기
n_feature=test_x_array.shape[2]
input_dim = n_feature
hidden_dim = 32
num_layers = 1
output_dim = n_feature
num_epochs = 7

In [8]:
## backward 실행 가능하도록 수정본
class LSTM(nn.Module):
  def __init__(
      self,
      input_dim,
      hidden_dim,
      num_layers,
      output_dim):
    
    self.input_dim=input_dim
    self.hidden_dim=hidden_dim
    self.num_layers=num_layers
    self.output_dim=output_dim

    super().__init__()

    self.lstm=nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True, bidirectional=True) 
    self.fc = nn.Linear(hidden_dim*2, output_dim)
    self.outputs=[]
  
  def forward(self, x, data_raw, lookback): #x =x_train, data_raw=x_series, lookback=주기
    outputs=[]
    for i, input_t in enumerate(x.chunk(x.size(0), dim=0)): #i=(1,lookback-1,n_feature)
      # |x|=(batch_size, time_step, vec_size)
      x,_=self.lstm(input_t)

      #|x|=(batch_size, time_step, hidden_size*2)
      y= self.fc(x[:,-1])
   
      #|y|=(batch_size, output_dim=n_feature)
      outputs += [y] #outputs : [[1], [2], ...] 형태


    #예측값을 입력값으로 넣을 준비
    future=120
    data_raw = data_raw.to_numpy() # df convert to numpy array

    #예측값을 입력값으로 넣어주기
    for index in range(len(data_raw)-lookback+1,len(data_raw)+future-lookback+1): #range(1361, 1481)
      if index<len(data_raw):
        idx=(index-(len(data_raw)-lookback+1))
        # 1361~1379 넣고 1380 예측으로 해야함
        if idx==0:
          train_ft=torch.from_numpy(data_raw[index:len(data_raw)]).type(torch.Tensor)
          train_ft=train_ft.view([-1,lookback-1,input_dim])

          # |x|=(batch_size, time_step, vec_size)
          train_ft, _=self.lstm(train_ft)

          #|x|=(batch_size, time_step, hidden_size*2)
          y= self.fc(train_ft[:,-1])

          #|y|=(batch_size, value)
          outputs += [y]  #outputs : [[1], [2], ...] 형태


        # 기존의 값들과 예측값을 혼재한 경우
        else:
          a=torch.from_numpy(data_raw[index:len(data_raw)]).type(torch.Tensor) # (lookback-1, input_dim=feature_n)
          b=torch.cat(outputs[-idx:], dim=0).view([-1,input_dim]) # (lookback-1,  input_dim=feature_n)
          train_ft=torch.cat([a,b],dim=0).view([-1,lookback-1,input_dim])

          # |x|=(batch_size, time_step, vec_size)
          train_ft, _=self.lstm(train_ft)

          #|x|=(batch_size, time_step, hidden_size*2)
          y= self.fc(train_ft[:,-1])

          #|y|=(batch_size, value)
          outputs += [y]  #outputs : [[1], [2], ...] 형태

      # 예측값들로만 이루어진 경우
      else:
        idx=lookback-1
        train_ft=torch.cat(outputs[-idx:], dim=0).view([-1,lookback-1,input_dim])

        # |x|=(batch_size, time_step, vec_size)
        train_ft, _=self.lstm(train_ft)

        #|x|=(batch_size, time_step, hidden_size*2)
        y= self.fc(train_ft[:,-1])

        #|y|=(batch_size, value)
        outputs += [y]  #outputs : [[1], [2], ...] 형태

    outputs = torch.stack(outputs,1).squeeze(0) #list형태를 torch 형태로 바꿔주기, (1361,output_dim=n_feature)
    ##outputs = torch.as_tensor(outputs) list형태를 torch 형태로 바꿔주는 것을 다음 코드로 했다가 loss.backward, optimizer.step이 작동하지 않았음
    #{outputs]=(batch_size, value)


    return outputs 

In [9]:
PATH='/content/drive/MyDrive/Colab Notebooks/Bitcoin_Trading'

model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers).to(device)
param=torch.load(PATH+'/100n_1L_BILSTM_7ep_5feature.pth')
model.load_state_dict(param['model_state_dict'])
model.eval()

LSTM(
  (lstm): LSTM(5, 32, batch_first=True, bidirectional=True)
  (fc): Linear(in_features=64, out_features=5, bias=True)
)

## 하나씩 sample_id를 가져와서 예측하기

In [10]:
import time
from tqdm import tqdm_notebook

result_pred_array=np.zeros([test_x_array.shape[0],120])
start_time=time.time()

for idxx in tqdm_notebook(range(test_x_array.shape[0])): #test_x_array.shape[0]

  n_feature=test_x_array.shape[2]
  t_series=test_x_array[idxx,:,:]

  t_series=pd.DataFrame(t_series)

  xt_train, yt_train = split_data(t_series, lookback) #lstm input 값 만들기

  xt_train = torch.from_numpy(xt_train).type(torch.Tensor) #(batch_size, time_seq,n_feature)=(1361,19,5)

  with torch.no_grad():
    #xt_train, t_series = xt_train.to(device), t_series.to(device)
    #t_series가 df이기 때문에 to(device)가 안됨.numpy일 떄도 안 됨. 즉, torch여야 하나봄.
    yt_train_pred = model(x=xt_train, data_raw=t_series, lookback=lookback) 
  
  yt_final=yt_train_pred.view([-1,n_feature]).detach().numpy()[:120,1] #(1340,5)->(120,), torch를 numpy로 바꿔주기
  yt_final=yt_final.reshape([-1, 120]) #(120,)->(1,120)

  result_pred_array[idxx,:]=yt_final

pred_time = time.time()-start_time
print("Pred time: {}".format(pred_time))


Pred time: 1596.7062113285065


## submission 파일(각 코인을 얼마나 언제 팔지)로 만들기
(sample_id, buy_quantity, sell_time) 

In [11]:
submission = array_to_submission(result_pred_array)
submission.head()

,sample_id,buy_quantity,sell_time
0,0,0.0,87
1,1,0.0,110
2,2,0.1,64
3,3,0.0,11
4,4,0.0,7


In [12]:
submission.buy_quantity.value_counts()

0.0    416
0.1    119
Name: buy_quantity, dtype: int64

In [20]:
submission.sample_id=submission.sample_id+7661

In [21]:
submission.to_csv(PATH+'/submisson_season2_1st.csv', sep=',', index=False)